In [1]:
import warnings
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense

2024-05-16 13:02:10.392512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 13:02:10.392701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 13:02:10.612596: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
test_df = pd.read_csv("/kaggle/input/data-storm-5-0/test_kaggle.csv")
test_df.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
0,33574,batticaloa,2686.5,3582,12537
1,10089,batticaloa,1717.56,2576.34,9446.58
2,38329,batticaloa,854.04,1242.24,5201.88
3,11376,batticaloa,1638.12,2320.67,9282.68
4,12410,batticaloa,1039.09,1518.67,5435.24


In [3]:
test_df.dtypes

Customer_ID      int64
outlet_city     object
luxury_sales    object
fresh_sales     object
dry_sales       object
dtype: object

In [4]:
numeric_column = pd.to_numeric(test_df['luxury_sales'], errors='coerce')
non_float_items = test_df[pd.isna(numeric_column)]
print(" Percentage of NaN values in luxury_sales column:"+ str((non_float_items.shape[0]/test_df.shape[0])*100))

 Percentage of NaN values in luxury_sales column:0.0049080959041939674


In [5]:
test_df.dropna(subset=['luxury_sales'], inplace=True)
numeric_column = pd.to_numeric(test_df['luxury_sales'], errors='coerce')
non_float_items = test_df[pd.isna(numeric_column)]
non_float_items

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
6,10364,batticaloa,Seven Hundread and sixty Ruppees,914.4,3352.8
6299,4059,Anuradhapura,Eight Hundread and fifteen ruppes,1548.69,5705.7


In [6]:
test_df.loc[6, 'luxury_sales'] = 760
test_df.loc[6299, 'luxury_sales'] = 815

In [7]:
test_df['luxury_sales'] = pd.to_numeric(test_df['luxury_sales'], errors='coerce')
test_df.dtypes

Customer_ID       int64
outlet_city      object
luxury_sales    float64
fresh_sales      object
dry_sales        object
dtype: object

In [8]:
numeric_column = pd.to_numeric(test_df['fresh_sales'], errors='coerce')
non_float_items = test_df[pd.isna(numeric_column)]
print(" Percentage of NaN values in fresh_sales column:"+ str((non_float_items.shape[0]/test_df.shape[0])*100))

 Percentage of NaN values in fresh_sales column:0.0024540479520969837


In [9]:
test_df.dropna(subset=['fresh_sales'], inplace=True)
numeric_column = pd.to_numeric(test_df['fresh_sales'], errors='coerce')
non_float_items = test_df[pd.isna(numeric_column)]
non_float_items

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
13949,28662,Jaffna,2195.84,Nine thousand ruppess,1509.64


In [10]:
test_df.loc[13949, 'fresh_sales'] = 9000

In [11]:
test_df['fresh_sales'] = pd.to_numeric(test_df['fresh_sales'], errors='coerce')
test_df.dtypes

Customer_ID       int64
outlet_city      object
luxury_sales    float64
fresh_sales     float64
dry_sales        object
dtype: object

In [12]:
numeric_column = pd.to_numeric(test_df['dry_sales'], errors='coerce')
non_float_items = test_df[pd.isna(numeric_column)]
print(" Percentage of NaN values in dry_sales column:"+ str((non_float_items.shape[0]/test_df.shape[0])*100))

 Percentage of NaN values in dry_sales column:0.0024540479520969837


In [13]:
test_df.dropna(subset=['dry_sales'], inplace=True)
numeric_column = pd.to_numeric(test_df['dry_sales'], errors='coerce')
non_float_items = test_df[pd.isna(numeric_column)]
non_float_items

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
40746,28664,Wattala,3838.6,3951.5,Three Thousand seven hundread ruppees


In [14]:
test_df.loc[40746, 'dry_sales'] = 3700

In [15]:
test_df['dry_sales'] = pd.to_numeric(test_df['dry_sales'], errors='coerce')
test_df.dtypes

Customer_ID       int64
outlet_city      object
luxury_sales    float64
fresh_sales     float64
dry_sales       float64
dtype: object

In [16]:
unique_values = test_df['outlet_city'].unique()
unique_values

array(['batticaloa', 'Batticaloa', 'Colombo', 'Dehiwala-Mount Lavinia',
       'Anuradhapura', 'Galle', 'Gampaha', 'Homagama', 'Jaffna',
       'Kaduwela', 'Kalmunai', 'kalmunai', 'Kandy', 'Katunayake',
       'Kelaniya', 'Madawachiya', 'Kotte', 'Moratuwa', 'MoraTuwa',
       'Negombo', 'Nuwara Eliya', 'Panadura', 'Peliyagoda', 'PeliyagodA',
       'Trincomale', 'Trincomalee', 'Wattala'], dtype=object)

In [17]:
test_df['outlet_city'] = test_df['outlet_city'].replace('MoraTuwa', 'Moratuwa')
test_df['outlet_city'] = test_df['outlet_city'].replace('PeliyagodA', 'Peliyagoda')
test_df['outlet_city'] = test_df['outlet_city'].replace('batticaloa', 'Batticaloa')
test_df['outlet_city'] = test_df['outlet_city'].replace('kalmunai', 'Kalmunai')
test_df['outlet_city'] = test_df['outlet_city'].replace('Trincomale', 'Trincomalee')

In [18]:
unique_values = test_df['outlet_city'].unique()
unique_values

array(['Batticaloa', 'Colombo', 'Dehiwala-Mount Lavinia', 'Anuradhapura',
       'Galle', 'Gampaha', 'Homagama', 'Jaffna', 'Kaduwela', 'Kalmunai',
       'Kandy', 'Katunayake', 'Kelaniya', 'Madawachiya', 'Kotte',
       'Moratuwa', 'Negombo', 'Nuwara Eliya', 'Panadura', 'Peliyagoda',
       'Trincomalee', 'Wattala'], dtype=object)

In [19]:
test_df.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
0,33574,Batticaloa,2686.50,3582.00,12537.00
1,10089,Batticaloa,1717.56,2576.34,9446.58
2,38329,Batticaloa,854.04,1242.24,5201.88
3,11376,Batticaloa,1638.12,2320.67,9282.68
4,12410,Batticaloa,1039.09,1518.67,5435.24
